<a href="https://colab.research.google.com/github/Duandi11/duandi11/blob/master/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/d2l-ai/d2l-zh@release  # installing d2l

In [4]:
%matplotlib inline
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from d2l import torch as d2l

In [36]:
batch_size, num_steps = 32,25
train_iter, vocab = d2l.load_data_time_machine(batch_size,num_steps)


In [37]:
X = torch.arange(10).reshape((2,5))

In [41]:
def get_params(vocab_size,num_hiddens,device):
    num_inputs = num_outputs = vocab_size
    
    def normal(shape):
        return torch.randn(size=shape,device=device) * 0.01
    W_xh = normal((num_inputs,num_hiddens))
    W_hh = normal((num_hiddens,num_hiddens))
    b_h = torch.zeros(num_hiddens,device=device)
    W_hq = normal((num_hiddens,num_outputs))
    b_q = torch.zeros(num_outputs,device=device)
    
    params = [W_xh,W_hh,b_h,W_hq,b_q]

    for param in params:
        param.requires_grad_(True)
    return params


def init_rnn_state(batch_size,num_hiddens,device):
    return (torch.zeros((batch_size,num_hiddens),device=device),)


def rnn(inputs,state,params):
    W_xh,W_hh,b_h,W_hq,b_q = params
    H, = state
    outputs = []

    for x in inputs:
        H = torch.tanh(torch.mm(x,W_xh)+torch.mm(H,W_hh)+b_h)
        Y = torch.mm(H,W_hq)+b_q
        outputs.append(Y)

    return torch.cat(outputs,dim=0),(H,)

In [39]:
class RNNModelScrath:
    def __init__(self,vocab_size,num_hiddens,device,
                 get_params,init_rnn_state,forward_fn):
        self.vocab_size = vocab_size
        self.num_hiddens = num_hiddens
        self.params = get_params(vocab_size,num_hiddens,device)
        self.init_state = init_rnn_state
        self.forward_fn = forward_fn

    def __call__(self,X,state):
        X = F.one_hot(X.T,self.vocab_size).type(torch.float32)
        return self.forward_fn(X,state,self.params)

    def begin_state(self,batch_size,device):
        return self.init_state(batch_size,self.num_hiddens,device)



In [42]:
num_hiddens = 512
net = RNNModelScrath(len(vocab),num_hiddens=num_hiddens,device='cpu',
                     get_params=get_params,init_rnn_state=init_rnn_state,forward_fn=rnn)
state = net.begin_state(X.shape[0],'cpu')
Y,new_state = net(X.to('cpu'),state)
Y.shape,len(new_state),new_state[0].shape

(torch.Size([10, 27]), 1, torch.Size([2, 512]))